In [52]:
#import libraries
import requests
from datetime import datetime as dt
from datetime import timedelta
import pandas as pd

In [53]:
# get events from n days ago
iso = 504 #Pike - changed the ISO code to download data for Morocco instead of Niger
limit = 400

api_url = 'https://api.acleddata.com/acled/read?terms=accept&iso={}'.format(iso)
print (api_url, type(api_url)) #creates request according to ACLED format specifications - p. 13

https://api.acleddata.com/acled/read?terms=accept&iso=504 <class 'str'>


In [54]:
response = requests.get(api_url)
data = response.json()
data.keys()
data['count']

500

### From the documentation we know this is the max return --- How can we get all the results?

In [55]:
# Let's mkae a function that updates our search to get the new pages

def ping_acled(api_url): 
    '''
    Takes one parameter search term for API
    '''
    
    response = requests.get(api_url)
    data = response.json()
    return data
    

In [56]:

results = [] # empty data strcture to store results
num_results = 500 # condition to continue adding pages
count = 0  # tracker of results
page = 1 #Per the documentation each page will give us more results

while num_results == 500: #if less 500 or 0 we know we have all the results
    print ("starting ", page, " ", num_results) #just to see our progress
    api_url = 'https://api.acleddata.com/acled/read?terms=accept&iso={}&page={}'.format(iso,page) #the search
    data = ping_acled(api_url) #call the previous function 
    results.append(data['data']) #store in our results
    count += data['count'] #Track number of results
    num_results = data['count'] #update our condition
    page += 1 #update our page variable
    print ("Total Results ", count) #Track our progress
    



starting  1   500
Total Results  500
starting  2   500
Total Results  1000
starting  3   500
Total Results  1500
starting  4   500
Total Results  2000
starting  5   500
Total Results  2500
starting  6   500
Total Results  3000
starting  7   500
Total Results  3070


In [57]:
#Now I want to put them together into one giant result
super_list = []
for res in results: 
    super_list += res
    print (len(super_list))

500
1000
1500
2000
2500
3000
3070


In [58]:
#convert it into an pandas data frame or just use your data structure and do more stuff
morocco_res = pd.DataFrame(super_list) #creating new pandas data frame for the Morocco data
morocco_res.head()

,data_id,iso,event_id_cnty,event_id_no_cnty,event_date,year,time_precision,event_type,sub_event_type,actor1,...,location,latitude,longitude,geo_precision,source,source_scale,notes,fatalities,timestamp,iso3
0,7227931,504,MOR3240,3240,2020-09-18,2020,1,Protests,Peaceful protest,Protesters (Morocco),...,El Kelaa des Sraghna,32.0512,-7.4108,1,Le Site Info,National,"On 18 September 2020, hundreds of merchants he...",0,1600717737,MAR
1,7227739,504,MOR3243,3243,2020-09-18,2020,1,Protests,Peaceful protest,Protesters (Morocco),...,Midelt,32.6811,-4.7296,2,Achkayen,National,"On 18 September 2020, 300 members of the Tanou...",0,1600717736,MAR
2,7228082,504,MOR3239,3239,2020-09-18,2020,1,Protests,Peaceful protest,Protesters (Morocco),...,Bounaamane,29.5313,-9.8135,1,Tiznit Press,Subnational,"On 18 September 2020, residents of the village...",0,1600717737,MAR
3,7227880,504,MOR3241,3241,2020-09-18,2020,2,Protests,Peaceful protest,Protesters (Morocco),...,Kenitra,34.2610,-6.5802,1,Hespress,National,"Around 18 September 2020 (as reported), reside...",0,1600717737,MAR
4,7227882,504,MOR3242,3242,2020-09-18,2020,1,Protests,Peaceful protest,Protesters (Morocco),...,Marrakech,31.6315,-8.0083,1,Hespress,National,"On 18 September 2020, the ""Regional Associatio...",0,1600717737,MAR


### Do the right thing, take some time to look at the codebook and see what these columns are

In [59]:
morocco_res.columns


Index(['data_id', 'iso', 'event_id_cnty', 'event_id_no_cnty', 'event_date',
       'year', 'time_precision', 'event_type', 'sub_event_type', 'actor1',
       'assoc_actor_1', 'inter1', 'actor2', 'assoc_actor_2', 'inter2',
       'interaction', 'region', 'country', 'admin1', 'admin2', 'admin3',
       'location', 'latitude', 'longitude', 'geo_precision', 'source',
       'source_scale', 'notes', 'fatalities', 'timestamp', 'iso3'],
      dtype='object')

### Homework --- Make a map of some ACLED Data (absolutely use the code from the Global Terrorism Database exercise)

In [62]:
#Imports necessary aspects of Bokeh for plotting on a map
from bokeh.tile_providers import get_provider, Vendors
from pyproj import Transformer
tile_provider = get_provider('STAMEN_TERRAIN')
import math ##Pike - we will need to import math and some bokeh.plotting tools
from bokeh.plotting import figure, output_notebook, show #builds interactive graphs for python

In [63]:
# Take the data reduced to a country and get the lat/long of the attacks and the name of the group
morocco_map = morocco_res[["latitude", 'longitude', 'data_id']]

In [64]:
#PIKE - we need to convert the data in the Pandas data frame from strings to floats for mapping
#we will get an error, but the code will still run

morocco_map['latitude'] = morocco_map['latitude'].astype(float)
morocco_map['longitude'] = morocco_map['longitude'].astype(float)
morocco_map['data_id'] = morocco_map['data_id'].astype(float)


<ipython-input-64-5f9c7551347e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  morocco_map['latitude'] = morocco_map['latitude'].astype(float)
<ipython-input-64-5f9c7551347e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  morocco_map['longitude'] = morocco_map['longitude'].astype(float)
<ipython-input-64-5f9c7551347e>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation

In [65]:
#create pyproj transformer to convert form lat/long to web mercator
transformer = Transformer.from_crs('epsg:4326','epsg:3857')

In [66]:
#Pike - changed the names to match the niger_map data frame

map_dict = {} # empty dictionary to track group attacks by lat long
nan_count = {} # some data doesn't have a lat/long so we need to know what we are losing


# Iterate through tables and associate group with lat/long
for idx, row in morocco_map.iterrows():    
    if row['data_id'] in map_dict.keys(): 
        if math.isnan(row["latitude"]):
            #This counts no data
            if row['data_id'] in nan_count.keys(): 
                nan_count[row['data_id']] += 1 
            else: 
                nan_count[row['data_id']] = 1
        else: 
            #This has to convert the lat/long to a mercator projection 
            point = transformer.transform(row["latitude"],row["longitude"])
            map_dict[row['data_id']].append([point[0],point[1]]) 
    #BOTH the if an else statement do the same thing but since it is a dictionary one needs to add the group name first
    else:  
        if math.isnan(row["latitude"]):
          nan_count[row['data_id']] = 1
        else: 
            point = transformer.transform(row["latitude"],row["longitude"])
            map_dict[row['data_id']] =[[point[0],point[1]]]
        
#This tells how many attacks we are losing
nan_count    

{}

In [68]:
pts = [(27.4,-13.5), (36.1, -0.9)] #Pike - ensured this bounding box encapsulated all of Morocco
bbox = []
for pt in transformer.itransform(pts): 
    bbox.append(pt)


In [69]:
NPA_x = []
NPA_y = []
for k,v in map_dict.items(): 
    for pt in v:
        NPA_x.append(pt[0])
        NPA_y.append(pt[1])

In [70]:
#Plots the bounding box
p = figure(x_range=(bbox[0][0], bbox[1][0]),y_range=(bbox[0][1], bbox[1][1]),x_axis_type="mercator", y_axis_type="mercator")
#add the map form the Bokeh map vendor in this case Stamen_Terrain --- see documentation
p.add_tile(tile_provider)
# Places a circle for each converted lat/long attack 
p.circle(x = NPA_x, y = NPA_y, color= "firebrick")

#shows the plot
show(p)

In [ ]:
##PIKE - running all this new code will correctly map the ACLED data for Morocco